In [1]:
import numpy as np
from scipy import stats
import pandas as pd

from data import DataFields3D, Dataset

In [2]:
# data from http://kdd.ics.uci.edu/databases/el_nino/el_nino.html

data_link = 'http://kdd.ics.uci.edu/databases/el_nino/elnino.gz'
data_desc = 'http://kdd.ics.uci.edu/databases/el_nino/elnino.col'

In [3]:
cols = pd.read_csv(data_desc, header=None, squeeze=True).to_list()
cols

['buoy',
 'day',
 'latitude',
 'longitude',
 'zon.winds',
 'mer.winds',
 'humidity',
 'air temp.',
 's.s.temp.']

In [4]:
data = (
    pd.read_csv(data_link, header=None, delim_whitespace=True, names=cols)
    .drop_duplicates(keep='last', subset=['buoy', 'day'])
    .query('humidity != "."')
    .astype({'humidity': float})
    .assign(z_himidity = lambda x: stats.zscore(x['humidity'].values))
)
data.head()

,buoy,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.,z_himidity
0,1,1,8.96,-140.32,-6.3,-6.4,83.5,27.32,27.57,-0.187797
1,1,2,8.95,-140.32,-5.7,-3.6,86.4,26.70,27.62,0.380607
2,1,3,8.96,-140.32,-6.2,-5.8,83.0,27.36,27.68,-0.285797
3,1,4,8.96,-140.34,-6.4,-5.3,82.2,27.32,27.70,-0.442598
4,1,5,8.96,-140.33,-4.9,-6.2,87.3,27.09,27.85,0.557008


In [5]:
data_fields = DataFields3D("latitude", "longitude", "day", weights='z_himidity')
data_fields

DataFields3D(x='latitude', y='longitude', z='day', weights='z_himidity')

In [6]:
dataset = Dataset(data, data_fields, name='El Nino')

In [7]:
with dataset.format('spatio_temporal_tensor'):
    *tensor_data, data_index = dataset.data

In [11]:
idx, vals, shape = tensor_data
print(f'Tensor with shape {shape} density: {idx.shape[0] / np.prod(shape):.1%}')

Tensor with shape (84, 104, 14) density: 0.5%


In [9]:
from tensor import sa_hooi

In [12]:
factors = sa_hooi(idx, vals, shape, (10, 12, 3))

growth of the core: 1.0
growth of the core: 0.12584701598390352
growth of the core: 0.010676055816409718
growth of the core: 0.017117682487169873
growth of the core: 0.013569693939591962
growth of the core: 0.007229069355844014
growth of the core: 0.0037107881377932946
growth of the core: 0.0010300888458770051
growth of the core: 0.0002229226605346933
Core is no longer growing. Norm of the core: 15.9249799630294.
